In [1]:
# Importing required libraries for machine learning
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the dataset
def load_dataset(file_path):
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'], utc=True)
    data['Timestamp'] = data['Date'].map(pd.Timestamp.timestamp)
    return data

In [3]:
# Prepare the data for regression (price prediction)
def prepare_regression_data(data):
    X = data[['Timestamp']].values
    y = data['Close'].values
    return train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Train regression model
def train_regression_model(data):
    X_train, X_test, y_train, y_test = prepare_regression_data(data)
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model

In [5]:
# Prepare data for classification (Buy/Sell/Hold)
def prepare_classification_data(data):
    data['Target'] = np.where(data['Close'].pct_change() > 0.02, 1,  # Buy
                              np.where(data['Close'].pct_change() < -0.02, -1, 0))  # Sell / Hold
    data.dropna(inplace=True)
    features = data[['Open', 'High', 'Low', 'Volume']].values
    labels = data['Target'].values
    scaler = StandardScaler()
    features = scaler.fit_transform(features)
    return train_test_split(features, labels, test_size=0.2, random_state=42)

In [6]:
# Train classification model
def train_classification_model(data):
    X_train, X_test, y_train, y_test = prepare_classification_data(data)
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)
    return clf

In [7]:
# Predict future price and recommendation
def predict_future(data, ticker, regression_model, classification_model):
    stock_data = data[data['Ticker'].str.upper() == ticker.upper()]
    if stock_data.empty:
        return {"error": "Stock not found"}

    # Get the latest stock data
    latest_stock = stock_data.iloc[-1:]
    future_price = regression_model.predict([[latest_stock['Timestamp'].values[0]]])[0]
    decision = classification_model.predict(latest_stock[['Open', 'High', 'Low', 'Volume']].values)[0]

    # Map decision to Buy/Sell/Hold
    decision_map = {-1: "Sell", 0: "Hold", 1: "Buy"}
    recommendation = decision_map.get(decision, "Hold")

    return {
        "future_price": round(future_price, 2),
        "recommendation": recommendation
    }

In [8]:
# Load dataset
file_path = "stock_data_5_years.csv"
data = load_dataset(file_path)

# Train models
regression_model = train_regression_model(data)
classification_model = train_classification_model(data)

# Example prediction
ticker = input("Enter the stock ticker symbol: ").strip()
result = predict_future(data, ticker, regression_model, classification_model)

if "error" in result:
    print(result["error"])
else:
    print(f"Future Price: ${result['future_price']}")
    print(f"Recommendation: {result['recommendation']}")

Enter the stock ticker symbol:  NFLX


Future Price: $227.99
Recommendation: Sell
